# Setup AWS EventBridge To Trigger Our Pipeline

In [3]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
import json
from botocore.exceptions import ClientError

sess   = sagemaker.Session()
bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
account_id = boto3.client('sts').get_caller_identity().get('Account')

## Steps
1. Create S3 Buckets
2. Enable CloudTrail Logging
3. Get StepFunctions Pipeline
4. Create EventBridge Rule
5. Test Trigger

### 1. Create S3 Data Upload Bucket (watched) & S3 Bucket for CloudTrail logs

In [4]:
watched_bucket = 'dsoaws-data-upload-{}'.format(account_id)
print(watched_bucket)

dsoaws-data-upload-806570384721


In [5]:
!aws s3 mb s3://$watched_bucket

make_bucket: dsoaws-data-upload-806570384721


In [6]:
!aws s3 ls $watched_bucket

# ^^^ GO TO S3 BUCKET in Console, click on Properties, Object-level logging, and click `enable` ^^^

In [8]:
cloudtrail_bucket = 'cloudtrail-dsoaws-{}'.format(account_id)
print(cloudtrail_bucket)

cloudtrail-dsoaws-806570384721


In [9]:
!aws s3 mb s3://$cloudtrail_bucket

make_bucket: cloudtrail-dsoaws-806570384721


In [10]:
!aws s3 ls $cloudtrail_bucket

                           PRE AWSLogs/


### Attach the following S3 policy to the Cloud Trail logging bucket created above ^^

In [11]:
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AWSCloudTrailAclCheck20150319",
            "Effect": "Allow",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:GetBucketAcl",
            "Resource": "arn:aws:s3:::{}".format(cloudtrail_bucket)
        },
        {
            "Sid": "AWSCloudTrailWrite20150319",
            "Effect": "Allow",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:PutObject",
            "Resource": "arn:aws:s3:::{}/AWSLogs/{}/*".format(cloudtrail_bucket, account_id),
            "Condition": {
                "StringEquals": {
                    "s3:x-amz-acl": "bucket-owner-full-control"
                }
            }
        },
        {
            "Sid": "AWSCloudTrailHTTPSOnly20180329",
            "Effect": "Deny",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:*",
            "Resource": [
                "arn:aws:s3:::{}/AWSLogs/{}/*".format(cloudtrail_bucket, account_id),
                "arn:aws:s3:::{}".format(cloudtrail_bucket)
            ],
            "Condition": {
                "Bool": {
                    "aws:SecureTransport": "false"
                }
            }
        }
    ]
}

print(policy)

{'Version': '2012-10-17', 'Statement': [{'Sid': 'AWSCloudTrailAclCheck20150319', 'Effect': 'Allow', 'Principal': {'Service': 'cloudtrail.amazonaws.com'}, 'Action': 's3:GetBucketAcl', 'Resource': 'arn:aws:s3:::cloudtrail-dsoaws-806570384721'}, {'Sid': 'AWSCloudTrailWrite20150319', 'Effect': 'Allow', 'Principal': {'Service': 'cloudtrail.amazonaws.com'}, 'Action': 's3:PutObject', 'Resource': 'arn:aws:s3:::cloudtrail-dsoaws-806570384721/AWSLogs/806570384721/*', 'Condition': {'StringEquals': {'s3:x-amz-acl': 'bucket-owner-full-control'}}}, {'Sid': 'AWSCloudTrailHTTPSOnly20180329', 'Effect': 'Deny', 'Principal': {'Service': 'cloudtrail.amazonaws.com'}, 'Action': 's3:*', 'Resource': ['arn:aws:s3:::cloudtrail-dsoaws-806570384721/AWSLogs/806570384721/*', 'arn:aws:s3:::cloudtrail-dsoaws-806570384721'], 'Condition': {'Bool': {'aws:SecureTransport': 'false'}}}]}


In [16]:
policy_json = json.dumps(policy)

In [18]:
with open("policy.json", "w") as outfile: 
    json.dump(policy, outfile)

In [24]:
!aws s3api put-bucket-policy --bucket $cloudtrail_bucket --policy file://policy.json

### Create CloudTrail and enable logging

In [12]:
!aws cloudtrail create-trail --name dsoaws --s3-bucket-name $cloudtrail_bucket --is-multi-region-trail

{
    "Name": "dsoaws",
    "S3BucketName": "cloudtrail-dsoaws-806570384721",
    "IncludeGlobalServiceEvents": true,
    "IsMultiRegionTrail": true,
    "TrailARN": "arn:aws:cloudtrail:us-east-1:806570384721:trail/dsoaws",
    "LogFileValidationEnabled": false,
    "IsOrganizationTrail": false
}


In [13]:
!aws cloudtrail start-logging --name dsoaws

In [14]:
!aws cloudtrail get-trail-status --name dsoaws

{
    "IsLogging": true,
    "StartLoggingTime": 1589136699.173,
    "LatestDeliveryAttemptTime": "",
    "LatestNotificationAttemptTime": "",
    "LatestNotificationAttemptSucceeded": "",
    "LatestDeliveryAttemptSucceeded": "",
    "TimeLoggingStarted": "2020-05-10T18:51:39Z",
    "TimeLoggingStopped": ""
}


## Create Custom EventBridge EventBus

In [40]:
events = boto3.client('events')

In [28]:
try:
    response = events.create_event_bus(Name='dsoaws')
    print(response)
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
        print("You're good. Event Bus already exists.")
    else:
        print("Unexpected error: %s" % e)

You're good. Event Bus already exists.


In [32]:
response = events.describe_event_bus(Name='dsoaws')
eventbus_arn = response['Arn']
print(eventbus_arn)

arn:aws:events:us-east-1:806570384721:event-bus/dsoaws


## Create Custom EventBridge Rule

In [33]:
pattern = {
  "source": [
    "aws.s3"
  ],
  "detail-type": [
    "AWS API Call via CloudTrail"
  ],
  "detail": {
    "eventSource": [
      "s3.amazonaws.com"
    ],
    "eventName": [
      "PutObject"
    ],
    "requestParameters": {
      "bucketName": [
        "{}".format(watched_bucket)
      ]
    }
  }
}

print(pattern)

{'source': ['aws.s3'], 'detail-type': ['AWS API Call via CloudTrail'], 'detail': {'eventSource': ['s3.amazonaws.com'], 'eventName': ['PutObject'], 'requestParameters': {'bucketName': ['dsoaws-data-upload-806570384721']}}}


In [35]:
pattern_json = json.dumps(pattern)

In [37]:
response = events.put_rule(
    Name='S3-Trigger',
    EventPattern=pattern_json,
    State='ENABLED',
    Description='Triggers an event on S3 PUT',
    EventBusName='dsoaws'
)
print(response)

{'RuleArn': 'arn:aws:events:us-east-1:806570384721:rule/dsoaws/S3-Trigger', 'ResponseMetadata': {'RequestId': '9f5683cf-f36b-45f8-8b00-dfcc1ce92700', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9f5683cf-f36b-45f8-8b00-dfcc1ce92700', 'content-type': 'application/x-amz-json-1.1', 'content-length': '74', 'date': 'Sun, 10 May 2020 19:10:27 GMT'}, 'RetryAttempts': 0}}


In [38]:
rule_arn = response['RuleArn']
print(rule_arn)

arn:aws:events:us-east-1:806570384721:rule/dsoaws/S3-Trigger


# Add Target

## TODO: Create IAM Role
`arn:aws:iam::806570384721:role/service-role/Amazon_EventBridge_Invoke_Step_Functions_Antje`  
Linking to StepFunctions Arn

In [ ]:
iam = boto3.client('iam')

In [46]:
iam_role_name_eventbridge = 'DSOAWS_EventBridge_Invoke_StepFunctions'

### Create AssumeRolePolicyDocument

In [41]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "events.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [42]:
try:
    iam_role_eventbridge = iam.create_role(
        RoleName=iam_role_name_eventbridge,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS EventBridge Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

### Get the Role ARN

In [50]:
role = iam.get_role(RoleName=iam_role_name_eventbridge)
iam_role_eventbridge_arn = role['Role']['Arn']
print(iam_role_eventbridge_arn)

arn:aws:iam::806570384721:role/DSOAWS_EventBridge_Invoke_StepFunctions


In [53]:
#stepfunctions_arn = '<STEPFUNCTION_ARN_TO_TRIGGER>'
stepfunctions_arn = 'arn:aws:states:us-east-1:806570384721:stateMachine:MyWorkflow_Simple'

### Define Eventbridge Policy

In [55]:
eventbridge_sfn_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "states:StartExecution"
            ],
            "Resource": [
                "{}".format(stepfunctions_arn)
            ]
        }
    ]
}
print(eventbridge_sfn_policy)

{'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Action': ['states:StartExecution'], 'Resource': ['arn:aws:states:us-east-1:806570384721:stateMachine:MyWorkflow_Simple']}]}


### Create Policy Object

In [56]:
try:
    policy_eventbridge_sfn = iam.create_policy(
      PolicyName='DSOAWS_EventBridgeInvokeStepFunction',
      PolicyDocument=json.dumps(eventbridge_sfn_policy)
    )
    print("Done.")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

Done.


### Get ARN

In [57]:
policy_eventbridge_sfn_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_EventBridgeInvokeStepFunction'
print(policy_eventbridge_sfn_arn)

arn:aws:iam::806570384721:policy/DSOAWS_EventBridgeInvokeStepFunction


### Attach Policy To Role

In [58]:
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_eventbridge_sfn_arn,
        RoleName=iam_role_name_eventbridge
    )
    print("Done.")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

Done.


## Create EventBridge Rule Target

In [60]:
sfn = boto3.client('stepfunctions')

In [ ]:
# ID: Id962e3164-fd3f-41a0-bb3d-47a1280f76fe

In [83]:
response = events.put_targets(
    Rule='S3-Trigger',
    EventBusName='dsoaws',
    Targets=[
        {
            'Id': 'Id962e3164-fd3f-41a0-bb3d-47a1280f76fe',
            'Arn': stepfunctions_arn,
            'RoleArn': iam_role_eventbridge_arn,
        }
    ]
)

In [84]:
print(response)

{'FailedEntryCount': 0, 'FailedEntries': [], 'ResponseMetadata': {'RequestId': '66c7eaa2-a1ff-45ec-bdb8-2c11f94569ea', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '66c7eaa2-a1ff-45ec-bdb8-2c11f94569ea', 'content-type': 'application/x-amz-json-1.1', 'content-length': '41', 'date': 'Sun, 10 May 2020 19:44:31 GMT'}, 'RetryAttempts': 0}}


## Check Stepfunctions - number of invocations

In [85]:
response = sfn.list_executions(
    stateMachineArn=stepfunctions_arn)
number = len(response['executions'])
print(number)

7


## Test: Upload a file into the S3 bucket

In [91]:
!aws s3 cp  ./src/requirements.txt s3://$watched_bucket

upload: src/requirements.txt to s3://dsoaws-data-upload-806570384721/requirements.txt


In [94]:
response = sfn.list_executions(
    stateMachineArn=stepfunctions_arn)
number = len(response['executions'])
print(number)

7


# DONE :-)